<h2/> Import all the necessary libraries. </h2>

In [3]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from math import cos, acos

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library
import itertools
import matplotlib.cm as cm
import matplotlib.colors as colors
from folium.features import DivIcon

<h3/>Center of Montréal island, which is the region under scope.</h3>

In [23]:
name = 'name'
lat_bus = 45.498398
long_bus = -73.578030
lista = [name, lat_bus, long_bus]
df = pd.DataFrame(columns = ('name', 'latitude', 'longitude'))
df.loc[0]=lista

In [24]:
df

,name,latitude,longitude
0,name,45.498398,-73.57803


In [28]:
CLIENT_ID = 'KRW3QNWYRXHGP4ETH1TQTSCVTNWJCXJBO2KB1TE0NNWUYGEG' 
CLIENT_SECRET = 'F40H3LLEQEJKCHCJT2GNRTSIDAJGIWVS4PKHDQWYSPLGCECB' 
VERSION = '20180604'
radius = 2000
LIMIT = 100
categoryId = '4d4b7105d754a06374d81259'

In [34]:
# for all the neighbourhoods
def getNearbyVenues(names, latitudes, longitudes, categoryId, radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL:
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            categoryId,
            LIMIT)
            
        # make the GET request:
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['name', 
                  'business Latitude', 
                  'business Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
# type your answer here

montreal_venues = getNearbyVenues(names=df['name'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude'],
                                   categoryId=categoryId,
                                   radius=radius
                                  
                                  )
print(montreal_venues.shape)
montreal_venues.head()



name
(100, 7)


,name,business Latitude,business Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,name,45.498398,-73.57803,La Panthère Verte,45.497335,-73.580144,Vegetarian / Vegan Restaurant
1,name,45.498398,-73.57803,Maison Boulud,45.500062,-73.578324,French Restaurant
2,name,45.498398,-73.57803,Mandy's,45.498286,-73.577895,Salad Place
3,name,45.498398,-73.57803,Leaves Cafe,45.498962,-73.577249,Café
4,name,45.498398,-73.57803,Café Aunja,45.497784,-73.580416,Café


In [130]:
print('There are {} uniques categories.'.format(len(montreal_venues['Venue Category'].unique())))

There are 185 uniques categories.


Let's use one hot encoding to calculate the frequency.

In [36]:
# one hot encoding
montreal_onehot = pd.get_dummies(montreal_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
montreal_onehot['name'] = montreal_venues['name'] 

# move neighborhood column to the first column
fixed_columns = [montreal_onehot.columns[-1]] + list(montreal_onehot.columns[:-1])
montreal_onehot = montreal_onehot[fixed_columns]

montreal_onehot.head()

,name,BBQ Joint,Bagel Shop,Bakery,Bistro,Breakfast Spot,Burger Joint,Café,Deli / Bodega,Diner,...,Portuguese Restaurant,Poutine Place,Restaurant,Salad Place,Sandwich Place,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Vegetarian / Vegan Restaurant
0,name,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,name,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,name,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,name,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,name,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


Group by the business of interest.

In [37]:
montreal_grouped = montreal_onehot.groupby('name').mean().reset_index()
montreal_grouped

,name,BBQ Joint,Bagel Shop,Bakery,Bistro,Breakfast Spot,Burger Joint,Café,Deli / Bodega,Diner,...,Portuguese Restaurant,Poutine Place,Restaurant,Salad Place,Sandwich Place,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Vegetarian / Vegan Restaurant
0,name,0.03,0.01,0.02,0.01,0.01,0.04,0.15,0.01,0.01,...,0.03,0.01,0.09,0.03,0.03,0.03,0.01,0.01,0.01,0.03


Find the top 5 most frequent venues.

In [40]:
num_bottom_venues = 5

for hood in montreal_grouped['name']:
    print("----"+str(hood)+"----")
    temp = montreal_grouped[montreal_grouped['name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).tail(num_top_venues))
    print('\n')

----name----
                  venue  freq
30   Mexican Restaurant  0.01
31  Fried Chicken Joint  0.01
32           Bagel Shop  0.01
33    Indian Restaurant  0.01
34  Dumpling Restaurant  0.01




<b>Introduction where you discuss the business problem and who would be interested in this project.</b>

The restaurant business is a competitive market for anyone to enter. As price war rages on, profits of restaurant owners are compromised. They must ensure that the quality of their offering is superior to their competitors while keeping the cost down. This is hard to manage; therefore, we suggest to help restaurant owner find a niche product offering that has less competition in order to boost higher margins.


<b>Data where you describe the data that will be used to solve the problem and the source of the data.<b>
    
In order to do this, we will take data from Foursquare and search for the most unique offering.
    

<b>Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.<b>
    
We have searched for the most unique type of restaurant within a two kilometer radius from Mandy's in order to decide on the potential product offering.
    

<b>Results section where you discuss the results.<b>
    
The algorithm recommends Mexican as the food for the restaurant owner as this food category has the highest potential with least competition in downtown Montreal.
    
    
<b>Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.<b>
    
We definitely did not expect to find Mexican food as teh niche; it might be hard to integrate into Mandy's current product offering.
    
    
<b>Conclusion section where you conclude the report.<b>

In conclusion, we suggest Mandy's stick to their current offering but add a Mexican twist, e.g. Salsa Salad.


 